Thursday, November 9, 2023

[LoRA for token classification](https://huggingface.co/docs/peft/task_guides/token-classification-lora)

This all runs!

## LoRA for token classification

Low-Rank Adaptation (LoRA) is a reparametrization method that aims to reduce the number of trainable parameters with low-rank representations. The weight matrix is broken down into low-rank matrices that are trained and updated. All the pretrained model parameters remain frozen. After training, the low-rank matrices are added back to the original weights. This makes it more efficient to store and train a LoRA model because there are significantly fewer parameters.

This guide will show you how to train a [roberta-large](https://huggingface.co/roberta-large) model with LoRA on the [BioNLP2004 dataset](https://huggingface.co/datasets/tner/bionlp2004) for token classification.

### Setup

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from datasets import load_dataset
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import evaluate
import torch
import numpy as np

model_checkpoint = "roberta-large"
lr = 1e-3
batch_size = 16
num_epochs = 10

### Load dataset and metric

The BioNLP2004 dataset includes tokens and tags for biological structures like DNA, RNA and proteins. Load the dataset:

In [3]:
bionlp = load_dataset("tner/bionlp2004")

In [4]:
bionlp["train"][0]

{'tokens': ['Since',
  'HUVECs',
  'released',
  'superoxide',
  'anions',
  'in',
  'response',
  'to',
  'TNF',
  ',',
  'and',
  'H2O2',
  'induces',
  'VCAM-1',
  ',',
  'PDTC',
  'may',
  'act',
  'as',
  'a',
  'radical',
  'scavenger',
  '.'],
 'tags': [0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

The tags values are defined in the label ids [dictionary](https://huggingface.co/datasets/tner/bionlp2004#label-id). The letter that prefixes each label indicates the token position: B is for the first token of an entity, I is for a token inside the entity, and 0 is for a token that is not part of an entity.

In [5]:
{
    "O": 0,
    "B-DNA": 1,
    "I-DNA": 2,
    "B-protein": 3,
    "I-protein": 4,
    "B-cell_type": 5,
    "I-cell_type": 6,
    "B-cell_line": 7,
    "I-cell_line": 8,
    "B-RNA": 9,
    "I-RNA": 10,
}

{'O': 0,
 'B-DNA': 1,
 'I-DNA': 2,
 'B-protein': 3,
 'I-protein': 4,
 'B-cell_type': 5,
 'I-cell_type': 6,
 'B-cell_line': 7,
 'I-cell_line': 8,
 'B-RNA': 9,
 'I-RNA': 10}

Then load the [seqeval](https://huggingface.co/spaces/evaluate-metric/seqeval) framework which includes several metrics - precision, accuracy, F1, and recall - for evaluating sequence labeling tasks.

In [6]:
seqeval = evaluate.load("seqeval")

Now you can write an evaluation function to compute the metrics from the model predictions and labels, and return the precision, recall, F1, and accuracy scores:


In [7]:
label_list = [
    "O",
    "B-DNA",
    "I-DNA",
    "B-protein",
    "I-protein",
    "B-cell_type",
    "I-cell_type",
    "B-cell_line",
    "I-cell_line",
    "B-RNA",
    "I-RNA",
]

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

### Preprocess dataset

Initialize a tokenizer and make sure you set is_split_into_words=True because the text sequence has already been split into words. However, this doesn’t mean it is tokenized yet (even though it may look like it!), and you’ll need to further tokenize the words into subwords.

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

You’ll also need to write a function to:

1) Map each token to their respective word with the word_ids method.
2) Ignore the special tokens by setting them to -100.
3) Label the first token of a given entity.

In [9]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Use map to apply the tokenize_and_align_labels function to the dataset:

In [10]:
tokenized_bionlp = bionlp.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3856 [00:00<?, ? examples/s]

Finally, create a data collator to pad the examples to the longest length in a batch:

In [11]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

### Train

Now you’re ready to create a [PeftModel](https://huggingface.co/docs/peft/v0.6.0/en/package_reference/peft_model#peft.PeftModel). Start by loading the base roberta-large model, the number of expected labels, and the id2label and label2id dictionaries:

In [12]:
id2label = {
    0: "O",
    1: "B-DNA",
    2: "I-DNA",
    3: "B-protein",
    4: "I-protein",
    5: "B-cell_type",
    6: "I-cell_type",
    7: "B-cell_line",
    8: "I-cell_line",
    9: "B-RNA",
    10: "I-RNA",
}
label2id = {
    "O": 0,
    "B-DNA": 1,
    "I-DNA": 2,
    "B-protein": 3,
    "I-protein": 4,
    "B-cell_type": 5,
    "I-cell_type": 6,
    "B-cell_line": 7,
    "I-cell_line": 8,
    "B-RNA": 9,
    "I-RNA": 10,
}

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint, num_labels=11, id2label=id2label, label2id=label2id
)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Define the [LoraConfig](https://huggingface.co/docs/peft/v0.6.0/en/package_reference/tuners#peft.LoraConfig) with:

* task_type, token classification (TaskType.TOKEN_CLS)
* r, the dimension of the low-rank matrices
* lora_alpha, scaling factor for the weight matrices
* lora_dropout, dropout probability of the LoRA layers
* bias, set to all to train all bias parameters

💡 The weight matrix is scaled by lora_alpha/r, and a higher lora_alpha value assigns more weight to the LoRA activations. For performance, we recommend setting bias to None first, and then lora_only, before trying all.

In [13]:
peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False, r=16, lora_alpha=16, lora_dropout=0.1, bias="all"
)

Pass the base model and peft_config to the get_peft_model() function to create a PeftModel. You can check out how much more efficient training the PeftModel is compared to fully training the base model by printing out the trainable parameters:

In [14]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,855,499 || all params: 355,905,558 || trainable%: 0.521345890304978


From the 🤗 Transformers library, create a [TrainingArguments](https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/trainer#transformers.TrainingArguments) class and specify where you want to save the model to, the training hyperparameters, how to evaluate the model, and when to save the checkpoints:

In [15]:
training_args = TrainingArguments(
    output_dir="roberta-large-lora-token-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

Pass the model, TrainingArguments, datasets, tokenizer, data collator and evaluation function to the Trainer class. The Trainer handles the training loop for you, and when you’re ready, call train to begin!

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_bionlp["train"],
    eval_dataset=tokenized_bionlp["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [17]:
%%time
trainer.train()

# CPU times: user 13min 13s, sys: 11.3 s, total: 13min 24s
# Wall time: 13min 31s

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: robkayinto. Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.227700,0.190244,0.727273,0.773636,0.749738,0.937175
2,0.194700,0.166844,0.745209,0.826400,0.783708,0.944928
3,0.178900,0.166029,0.746623,0.786422,0.766006,0.943629
4,0.163700,0.158364,0.767941,0.824779,0.795346,0.948607
5,0.150100,0.155181,0.776735,0.834504,0.804584,0.949335
6,0.138000,0.162278,0.770978,0.837205,0.802728,0.947564
7,0.128900,0.153651,0.781043,0.828021,0.803846,0.949709
8,0.115400,0.153835,0.789286,0.835764,0.811860,0.950516
9,0.104500,0.153946,0.802551,0.827120,0.814651,0.951106
10,0.095100,0.155473,0.802936,0.837205,0.819713,0.951735


CPU times: user 13min 13s, sys: 11.3 s, total: 13min 24s
Wall time: 13min 31s


TrainOutput(global_step=10390, training_loss=0.15465380656707733, metrics={'train_runtime': 807.932, 'train_samples_per_second': 205.698, 'train_steps_per_second': 12.86, 'total_flos': 2.356355526584813e+16, 'train_loss': 0.15465380656707733, 'epoch': 10.0})

### Share model

Once training is complete, you can store and share your model on the Hub if you’d like. Log in to your Hugging Face account and enter your token when prompted:

In [18]:
from huggingface_hub import notebook_login

# notebook_login()

Upload the model to a specific model repository on the Hub with the push_to_hub method:

In [19]:
peft_model_id = "robkayinto/roberta-large-lora-token-classification"
model.push_to_hub(peft_model_id)

adapter_model.bin:   0%|          | 0.00/7.53M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/robkayinto/roberta-large-lora-token-classification/commit/32b36abe669b93a135eadfa38c79b8929faf3e5c', commit_message='Upload model', commit_description='', oid='32b36abe669b93a135eadfa38c79b8929faf3e5c', pr_url=None, pr_revision=None, pr_num=None)

### Inference

To use your model for inference, load the configuration and model:

In [20]:
config = PeftConfig.from_pretrained(peft_model_id)

inference_model = AutoModelForTokenClassification.from_pretrained(
    config.base_model_name_or_path, num_labels=11, id2label=id2label, label2id=label2id
)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(inference_model, peft_model_id)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Get some text to tokenize:

In [21]:
text = "The activation of IL-2 gene expression and NF-kappa B through CD28 requires reactive oxygen production by 5-lipoxygenase."
inputs = tokenizer(text, return_tensors="pt")

Pass the inputs to the model, and print out the model prediction for each token:

In [22]:
with torch.no_grad():
    logits = model(**inputs).logits

tokens = inputs.tokens()
predictions = torch.argmax(logits, dim=2)

for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))


('<s>', 'O')
('The', 'O')
('Ġactivation', 'O')
('Ġof', 'O')
('ĠIL', 'B-protein')
('-', 'I-DNA')
('2', 'I-DNA')
('Ġgene', 'O')
('Ġexpression', 'O')
('Ġand', 'O')
('ĠNF', 'B-protein')
('-', 'I-protein')
('k', 'I-protein')
('appa', 'I-protein')
('ĠB', 'I-protein')
('Ġthrough', 'O')
('ĠCD', 'B-protein')
('28', 'I-protein')
('Ġrequires', 'O')
('Ġreactive', 'O')
('Ġoxygen', 'O')
('Ġproduction', 'O')
('Ġby', 'O')
('Ġ5', 'B-protein')
('-', 'I-protein')
('lip', 'I-protein')
('oxy', 'I-protein')
('gen', 'I-protein')
('ase', 'I-protein')
('.', 'O')
('</s>', 'O')
